In [9]:
# 引用需要的library
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from matplotlib import pyplot as plt
import helper

In [31]:
# 从文本文件中读数据
# 读取到的内容
# X=评论（string）， y=情感分类（0/1）

import os
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
folder_path = './user_comments'  # 请将此处替换为您的文件夹路径
X = []
y = []
# 使用CountVectorizer进行文本向量化
vectorizer = CountVectorizer()
for root, dirs, files in os.walk(folder_path):
    for dir in dirs:
        if dir == 'neg':
            label = 0
        elif dir == 'pos':
            label = 1
        else:
            continue
        for file in os.listdir(os.path.join(folder_path, dir)):
            if file.endswith('.txt'):
                file_path = os.path.join(folder_path, dir, file)
                with open(file_path, 'r', encoding='gb2312',errors='ignore') as f:
                    content = f.read()
                    content = content.strip().replace(" ", "")
                    if content == "": break
                # 使用jieba进行分词
                words = ' '.join(jieba.cut(content))
                X.append(words)
                y.append(label)

X=np.array(X)
y=np.array(y)
# print(X.shape)
# (4905,)

X = vectorizer.fit_transform(X)

# 打印转化为词向量的X数据，是一个稀疏的矩阵(4905, 18300)
# print(X.toarray())
# print(X.shape)

# voca = vectorizer.vocabulary_
# print(voca)
# vocabulary dict

# 对dict重新排序，按照value的顺序打印dict
# voca=sorted(voca.items(), key=lambda x: x[1])
# for item in voca:
#     print(item)





X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# 调用jieba分词
# 文本向量化：1. 类one-hot编码 2. count思想 3. tf-idf思想
# 返回 X=评论向量（list），y=情感分类（0/1）

In [ ]:
# 切分数据集为X_train, y_train, X_test, y_test


In [ ]:
# 数据预处理：标准化/归一化

In [ ]:
# 调用各分类模型，训练
# KNeighborsClassifier
# DecisionTreeClassifier
# SVC
# RandomForestClassifier
# AdaBoostClassifier
# GradientBoostingClassifier
# XGBClassifier
# LGBMClassifier

In [ ]:
# 调用训练完毕的模型，做预测

In [ ]:
# 计算各模型的acc，训练时间，预测时间

In [ ]:
# 保存模型到models文件夹